# note: This program needs openAI API

# Prep dataset

In [1]:
import pandas as pd
import numpy as np
import re
import os
import scipy
from torch import tensor

os.getcwd()

'/Users/yancong/Desktop/4 clinical/02 projects_parsely/05 ssd-lm-stanglab/15 simulation/GPT3simul'

In [2]:
data_foloder = '/Gradient_shuffle/'
result = '/Analysis/'
baseline = pd.read_csv(data_foloder + 'simulation_HV_baseline_vb_response_deid_v3.csv')
incoh10 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_10v3.csv')
incoh20 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_20v3.csv')
incoh50 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_50v3.csv')
ineff10 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_10v3.csv')
ineff20 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_20v3.csv')
ineff50 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_50v3.csv')

baseline.head()

,grid,content,n_words
0,10455,"I'm a young man , an en an en- an engineer by ...",421
1,11689,Sure . I'm thirty three years old . My name is...,159
2,12376,Alright . um I live in not especially cool Spr...,468
3,12630,um So I'm currently twenty-nine . I was born a...,966
4,13493,Mhm . I'm a thirty five year old man who uh um...,134


# Install lib and dependencies

In [ ]:
!pip install transformers # no need to install every time you open it

In [3]:
import transformers
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [4]:
from openai.embeddings_utils import get_embeddings, cosine_similarity # important
import openai
openai.api_key = 'your key'

# Get sent embeddings from GPT3 response embeddings 3df

In [6]:
buglist = {}
def calc_gpt3(r): 
  if r[0] % 2 != 0: # progress
    print('current grid: ', r[0])
  if type(r['content']) == str and r['n_words'] < 2048: # max for GPT3
    temp = r['content'].split(' ') 
    input = ' '.join([i for i in temp if (i != '') & ('{' not in i) & ('}' not in i) & ('#' not in i)]) # exclude NSV and the restart symbol
    try:
      vec = get_embeddings(input.split(' . '), engine = 'text-similarity-babbage-001') # take in the whose response as a list of sentence, sep by .
    except:
      buglist[r[0]] = input.split(' ') # store grid
    return vec
  else:
    return np.nan

# double check your input df
# make sure there is no '  ' or '   '
# put together all the dataframes into a list
# to avoid repeated work
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["gpt3_embed"] = '' # create a new empty column
  df["gpt3_embed"] = df.apply(lambda r: calc_gpt3(r), axis = 1) # apply the embedding function to the dataframe
  df.to_csv(result + str(temp) + '_sent.csv')
df.head()

current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493


,grid,content,n_words,gpt3_embed
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.008276593871414661, -0.0036779942456632853..."
1,11689,We have been using that opportunity to do more...,159,"[[-0.0030545061454176903, -0.02546398527920246..."
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[0.009739612229168415, -0.038222722709178925,..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ..."


In [7]:
buglist

{}

# FOC and SOC

In [8]:
temp_file = -1 # keep track of file names
for df in dfs:
    temp_file += 1
    df['gpt3_foc'] = ''
    df['gpt3_soc'] = ''
    # loop over each response
    for i in df.index:
        # calculate average similarity for sentence pairs, either adjacent or with one intervening
        for idx, sent in enumerate(df['gpt3_embed'][i]):
            temp_foc = []
            temp_soc = []
            try:
                temp_foc.append(cosine_similarity(sent, df['gpt3_embed'][i][idx+1])) # get a list of similarities for that response
                df['gpt3_foc'][i] = np.average(temp_foc) # add more stats here

                temp_soc.append(cosine_similarity(sent, df['gpt3_embed'][i][idx+2]))
                df['gpt3_soc'][i] = np.average(temp_soc)
            except IndexError:
                continue
    df.to_csv(result + str(temp_file) + '_sent.csv')
df.head()

/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_26198/1417544554.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt3_foc'][i] = np.average(temp_foc)
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_26198/1417544554.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt3_soc'][i] = np.average(temp_soc)


,grid,content,n_words,gpt3_embed,gpt3_foc,gpt3_soc
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.008276593871414661, -0.0036779942456632853...",0.769683,0.77621
1,11689,We have been using that opportunity to do more...,159,"[[-0.0030545061454176903, -0.02546398527920246...",0.655993,0.610017
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ...",0.580082,0.633091
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[0.009739612229168415, -0.038222722709178925,...",0.747916,0.648297
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ...",0.669287,0.723936


# Clinical factor scores

In [9]:
tlc = pd.read_csv('/Users/yancong/Desktop/4 clinical/00 Project Files/crossdx_clin.csv', index_col=0)
tlc = tlc[['grid', 'SSDvHC', 'group', 'tlc_01povspeech', 'tlc_02povcontent',	'tlc_03pressure',	'tlc_04distract',
	'tlc_05tangent', 'tlc_06derail', 'tlc_07incoh',	'tlc_08illogic',	'tlc_09clang',	'tlc_10neologism',
    	'tlc_11wordapprox',	'tlc_12circum',	'tlc_13lossgoal',	'tlc_14persev',	'tlc_15echo',	'tlc_16block',
        	'tlc_17stilt',	'tlc_18selfref', 'tlc_3f_inefficient',	'tlc_3f_incoherent',	'tlc_3f_impexpress']]
tlc.head()

,grid,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,tlc_04distract,tlc_05tangent,tlc_06derail,tlc_07incoh,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
1,10308,1.0,SSD,0,1,0,0,0,0,0,...,0,0,1,0,0,2,0,-0.406404,-0.069358,-0.018896
2,10311,0.0,HC,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.713934,-0.463481,0.197262
3,10316,1.0,SSD,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,10455,0.0,HC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,10582,1.0,SSD,0,0,0,1,0,1,0,...,1,1,0,0,0,0,0,-0.059923,-0.107333,-0.458561


In [10]:
temp_file = -1
for df in dfs:
    temp_file += 1
    df['grid'] = df['grid'].astype(str)
    df = df.merge(tlc, on=['grid']) # merge with the clinical dataset
    del df['gpt3_embed']
    df.to_csv(result + str(temp_file) + '_sent.csv')
df.head()

,grid,content,n_words,gpt3_foc,gpt3_soc,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,10455,"I'm a young man , an en an en- an engineer by ...",421,0.769683,0.77621,0.0,HC,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,11689,We have been using that opportunity to do more...,159,0.655993,0.610017,0.0,HC,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,12376,Alright . um I live in not especially cool Spr...,468,0.580082,0.633091,0.0,HC,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,12630,"My is things are fantastic . No , I mean My uh...",966,0.747916,0.648297,0.0,HC,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,13493,Mhm . I still get to play . And my stock inves...,134,0.669287,0.723936,0.0,HC,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348


# Combine data frames

In [16]:
baseline_word = pd.read_csv('Analysis/0_GT.csv', index_col=0)
baseline_word['group'] = 'HV_0_shuffle'
baseline_word.drop('grid', axis = 1, inplace = True)

incoh10 = pd.read_csv('Analysis/1_GT.csv', index_col=0)
incoh10['SSDvHC'] = 0.1
incoh10['group'] = 'HV_10_shuffle'
incoh10.drop('grid', axis = 1, inplace = True)

incoh20 = pd.read_csv('Analysis/2_GT.csv', index_col=0)
incoh20['SSDvHC'] = 0.2
incoh20['group'] = 'HV_20_shuffle'
incoh20.drop('grid', axis = 1, inplace = True)

incoh50 = pd.read_csv('Analysis/3_GT.csv', index_col=0)
incoh50['SSDvHC'] = 0.5
incoh50['group'] = 'HV_50_shuffle'
incoh50.drop('grid', axis = 1, inplace = True)

incoh50.head()

,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10,gpt3_word_mv5,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,0.785865,0.792608,0.793065,0.788796,0.784576,0.784141,0.785274,0.787505,0.783662,0.783619,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,0.795350,0.786443,0.786314,0.788560,0.790149,0.785299,0.786570,0.799864,0.785689,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,0.785127,0.783258,0.784617,0.784282,0.782749,0.781022,0.782449,0.785904,0.780884,0.781878,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,0.790323,0.791585,0.787778,0.788561,0.790323,0.791137,0.795051,0.790728,0.788126,0.786324,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,0.784419,0.793565,0.790748,0.786931,0.781176,0.782685,0.794357,0.790361,0.791204,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348


In [17]:
incoh = pd.concat([baseline_word, incoh10, incoh20, incoh50])
incoh.reset_index(drop = True, inplace = True)
incoh.to_csv(result + 'GPT3_simul_vb_open_baseline_incoh_gt.csv')
incoh.head(20)

,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10,gpt3_word_mv5,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,0.785325,0.786946,0.789100,0.786143,0.781507,0.783395,0.778689,0.782849,0.778174,0.778152,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,0.796211,0.794249,0.792468,0.790037,0.795478,0.795065,0.790552,0.795920,0.795782,0.790783,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,0.788436,0.788807,0.789494,0.790119,0.785886,0.785951,0.785626,0.792179,0.787467,0.787950,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,0.792645,0.792898,0.789984,0.791605,0.792505,0.794107,0.796284,0.791136,0.791520,0.789007,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,0.802482,0.811824,0.801784,0.802693,0.799897,0.802248,0.805543,0.806474,0.813133,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,0.784172,0.787488,0.788652,0.785579,0.782508,0.782644,0.780111,0.783165,0.778421,0.777958,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
6,0.797041,0.794726,0.791040,0.790324,0.798740,0.790898,0.788882,0.800755,0.791009,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
7,0.788402,0.787513,0.788218,0.787090,0.785616,0.784563,0.785504,0.788903,0.785712,0.786147,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
8,0.791292,0.791553,0.788850,0.789650,0.791367,0.792138,0.794664,0.789987,0.790205,0.788010,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
9,0.801040,0.808678,0.802299,0.801240,0.798119,0.798631,0.806279,0.806024,0.812273,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348


In [18]:
ineff10 = pd.read_csv('Analysis/4_GT.csv', index_col=0)
ineff10['SSDvHC'] = 0.1
ineff10['group'] = 'HV_10_shuffle'
ineff10.drop('grid', axis = 1, inplace = True)

ineff20 = pd.read_csv('Analysis/5_GT.csv', index_col=0)
ineff20['SSDvHC'] = 0.2
ineff20['group'] = 'HV_20_shuffle'
ineff20.drop('grid', axis = 1, inplace = True)

ineff50 = pd.read_csv('Analysis/6_GT.csv', index_col=0)
ineff50['SSDvHC'] = 0.5
ineff50['group'] = 'HV_50_shuffle'
ineff50.drop('grid', axis = 1, inplace = True)

ineff = pd.concat([baseline_word, ineff10, ineff20, ineff50])
ineff.reset_index(drop = True, inplace = True)
ineff.to_csv(result + 'GPT3_simul_vb_open_baseline_ineff_gt.csv')
ineff.head(20)

,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10,gpt3_word_mv5,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,0.785325,0.786946,0.789100,0.786143,0.781507,0.783395,0.778689,0.782849,0.778174,0.778152,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,0.796211,0.794249,0.792468,0.790037,0.795478,0.795065,0.790552,0.795920,0.795782,0.790783,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,0.788436,0.788807,0.789494,0.790119,0.785886,0.785951,0.785626,0.792179,0.787467,0.787950,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,0.792645,0.792898,0.789984,0.791605,0.792505,0.794107,0.796284,0.791136,0.791520,0.789007,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,0.802482,0.811824,0.801784,0.802693,0.799897,0.802248,0.805543,0.806474,0.813133,NaN,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,0.786382,0.786675,0.790395,0.787486,0.781349,0.784620,0.780977,0.783304,0.778792,0.781592,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
6,0.792679,0.790801,0.787366,0.787827,0.788522,0.791426,0.785540,0.795135,0.789005,0.790117,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
7,0.787103,0.788427,0.786882,0.792280,0.784875,0.789110,0.785489,0.792105,0.787284,0.787215,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
8,0.794404,0.794834,0.792990,0.793617,0.793519,0.794366,0.795469,0.793102,0.791658,0.791781,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
9,0.800266,0.806206,0.796555,0.798258,0.795871,0.798677,0.800741,0.802697,0.809049,0.798662,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348


In [19]:
baseline_sent = pd.read_csv('Analysis/0_sent.csv', index_col=0)
baseline_sent['group'] = 'HV_0_shuffle'
baseline_sent.drop('grid', axis = 1, inplace = True)

incoh10 = pd.read_csv('Analysis/1_sent.csv', index_col=0)
incoh10['SSDvHC'] = 0.1
incoh10['group'] = 'HV_10_shuffle'
incoh10.drop('grid', axis = 1, inplace = True)

incoh20 = pd.read_csv('Analysis/2_sent.csv', index_col=0)
incoh20['SSDvHC'] = 0.2
incoh20['group'] = 'HV_20_shuffle'
incoh20.drop('grid', axis = 1, inplace = True)

incoh50 = pd.read_csv('Analysis/3_sent.csv', index_col=0)
incoh50['SSDvHC'] = 0.5
incoh50['group'] = 'HV_50_shuffle'
incoh50.drop('grid', axis = 1, inplace = True)

incoh = pd.concat([baseline_sent, incoh10, incoh20, incoh50])
incoh.reset_index(drop = True, inplace = True)
incoh.to_csv(result + 'GPT3_simul_vb_open_baseline_incoh_sent_gt.csv')
incoh.head(20)

,content,n_words,gpt3_foc,gpt3_soc,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,tlc_04distract,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,"I'm a young man , an en an en- an engineer by ...",421,0.745460,0.771513,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,Sure . I'm thirty three years old . My name is...,159,0.658458,0.682094,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,Alright . um I live in not especially cool Spr...,468,0.802208,0.792694,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,um So I'm currently twenty-nine . I was born a...,966,0.683852,0.703612,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,Mhm . I'm a thirty five year old man who uh um...,134,0.592401,0.688711,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,"I'm a young man , an en an en- an engineer by ...",421,0.745460,0.756312,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
6,Sure . I'm thirty three years good . My name i...,159,0.711289,0.682094,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
7,Alright . um I live in not especially cool Spr...,468,0.802208,0.809238,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
8,um So I'm currently twenty-nine . I was born a...,966,0.685615,0.696715,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
9,Mhm . I'm a thirty five year old man who uh um...,134,0.622101,0.699478,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348


In [20]:
ineff10 = pd.read_csv('Analysis/4_sent.csv', index_col=0)
ineff10['SSDvHC'] = 0.1
ineff10['group'] = 'HV_10_shuffle'
ineff10.drop('grid', axis = 1, inplace = True)

ineff20 = pd.read_csv('Analysis/5_sent.csv', index_col=0)
ineff20['SSDvHC'] = 0.2
ineff20['group'] = 'HV_20_shuffle'
ineff20.drop('grid', axis = 1, inplace = True)

ineff50 = pd.read_csv('Analysis/6_sent.csv', index_col=0)
ineff50['SSDvHC'] = 0.5
ineff50['group'] = 'HV_50_shuffle'
ineff50.drop('grid', axis = 1, inplace = True)

ineff = pd.concat([baseline_sent, ineff10, ineff20, ineff50])
ineff.reset_index(drop = True, inplace = True)
ineff.to_csv(result + 'GPT3_simul_vb_open_baseline_ineff_sent_gt.csv')
ineff.head(20)

,content,n_words,gpt3_foc,gpt3_soc,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,tlc_04distract,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,"I'm a young man , an en an en- an engineer by ...",421,0.745460,0.771513,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,Sure . I'm thirty three years old . My name is...,159,0.658458,0.682094,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,Alright . um I live in not especially cool Spr...,468,0.802208,0.792694,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,um So I'm currently twenty-nine . I was born a...,966,0.683852,0.703612,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,Mhm . I'm a thirty five year old man who uh um...,134,0.592401,0.688711,0.0,HV_0_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,"I'm a young man , an en an en- an engineer by ...",421,0.756359,0.771513,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
6,Sure . I'm thirty three years old . My name is...,159,0.736462,0.733960,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
7,Alright . um I live in not especially cool Spr...,468,0.802208,0.792694,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
8,um So I was born and raised in South Washingto...,966,0.747731,0.702150,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
9,Mhm . I'm a thirty five year old man who uh um...,134,0.676761,0.664068,0.1,HV_10_shuffle,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
